In [118]:
import pandas as pd
import numpy as np

In [119]:
#Load Data
temp_file = 'Resources/temp_data.csv'
rainfall_file = 'Resources/rainfall_data.csv'
temp_data = pd.read_csv(temp_file, error_bad_lines=False)
rainfall_data = pd.read_csv(rainfall_file, error_bad_lines=False)
happy_2015 = pd.read_csv('Resources/2015.csv')
happy_2016 = pd.read_csv('Resources/2016.csv')
happy_2017 = pd.read_csv('Resources/2017.csv')
happy_2018 = pd.read_csv('Resources/2018.csv')
happy_2019 = pd.read_csv('Resources/2019.csv')

In [161]:
#Clean & Merge weather data
#drop na values
temp_data = temp_data.dropna()
rainfall_data = rainfall_data.dropna()

#merge rainfall into temp dataframe
combined_weather_df = pd.merge(temp_data, rainfall_data, how="outer")

#add Degrees F Column
combined_weather_df["Temperature - F"] = combined_weather_df["Temperature - (Celsius)"]*9/5 + 32

#group by country & year
combined_data_groupby = combined_weather_df.groupby(["Country", "Year"])

#temp_data_groupby.head(10)
combined_data_groupby.mean()

Temperature - (Celsius)  Rainfall - (MM)  Temperature - F
Country      Year                                                           
 Afghanistan 1991                12.894488        36.287492        55.210078
             1992                12.544548        34.013019        54.580186
             1993                12.977138        26.423775        55.358849
             1994                13.080018        28.518532        55.544033
             1995                13.040895        25.074846        55.473611
...                                    ...              ...              ...
 Zimbabwe    2012                21.910075        45.312368        71.438135
             2013                21.624350        51.234424        70.923830
             2014                21.710483        50.608135        71.078870
             2015                22.327625        32.552881        72.189725
             2016                22.310167        37.927746        72.158300

[5070 rows x 3 columns]

In [169]:
#Clean & merge happiness data
#Add year column
happy_2015["Year"] = 2015
happy_2016["Year"] = 2016
happy_2017["Year"] = 2017
happy_2018["Year"] = 2018
happy_2019["Year"] = 2019

#Rename columns (Country, Happiness Rank, Happiness Score, Year)
happy_2017 = happy_2017.rename(columns={
    "Happiness.Rank":"Happiness Rank",
    "Happiness.Score":"Happiness Score"
})
happy_2018 = happy_2018.rename(columns={
    "Overall rank":"Happiness Rank",
    "Score":"Happiness Score",
    "Country or region":"Country"
})

happy_2019 = happy_2019.rename(columns={
    "Overall rank":"Happiness Rank",
    "Score":"Happiness Score",
    "Country or region":"Country"
})

happy_2015_clean = happy_2015[["Year","Country","Happiness Rank","Happiness Score"]]
happy_2016_clean = happy_2016[["Year","Country","Happiness Rank","Happiness Score"]]
happy_2017_clean = happy_2017[["Year","Country","Happiness Rank","Happiness Score"]]
happy_2018_clean = happy_2018[["Year","Country","Happiness Rank","Happiness Score"]]
happy_2019_clean = happy_2019[["Year","Country","Happiness Rank","Happiness Score"]]

#Merge datasets
combined_happy_df = pd.concat([happy_2015_clean, happy_2016_clean, happy_2017_clean, happy_2018_clean, happy_2019_clean], ignore_index=True)

happy_groupby = combined_happy_df.groupby(["Country", "Year"])
happy_groupby.mean()


Happiness Rank  Happiness Score
Country     Year                                 
Afghanistan 2015             153            3.575
            2016             154            3.360
            2017             141            3.794
            2018             145            3.632
            2019             154            3.203
...                          ...              ...
Zimbabwe    2015             115            4.610
            2016             131            4.193
            2017             138            3.875
            2018             144            3.692
            2019             146            3.663

[782 rows x 2 columns]